## Tacotron 2 inference code
Edit the variables **checkpoint_path** and **text** to match yours and run the entire code to generate plots of mel outputs, alignments and audio synthesis from the generated mel-spectrogram using Griffin-Lim.

#### Import liraries and setup matplotlib

In [3]:
from pydub import AudioSegment
from numpy import ndarray
from pathlib import Path
from typing import List
%matplotlib inline
import sys

from scipy.io.wavfile import write
import re
import matplotlib.pylab as plt

sys.path.append('nvidia_waveglow/')
import numpy as np
import torch

from hparams import create_hparams
from train import load_model
from libs.text import text_to_sequence
from nvidia_waveglow.denoiser import Denoiser

def plot_data(data, figsize=(16, 4)):
    fig, axes = plt.subplots(1, len(data), figsize=figsize)
    for i in range(len(data)):
        axes[i].imshow(data[i], aspect='auto', origin='lower',
                       interpolation='none')

def prepare_output_filepath(output_dir, sentence):
    filename = re.sub(r'[^a-ž ]', '', sentence.lower())
    filename = re.sub(r"[\u0300\u0301\u0303]", '', filename)
    filename = '_'.join(filename.split())
    return f"{output_dir}/{filename}.wav"

def prepare_sentences(text: str, max_len=500):
    text = re.sub(r'([!?.])\s{2,}', '\1\n', text)
    lines = text.split('\n')
    lines = [re.sub(r'\s{2,}', ' ', line) for line in lines]
    return [line.strip() for line in lines
            if line and len(line) < max_len]

def prepare_sequence(sentence: str):
    sequence = np.array(text_to_sequence(sentence, ['basic_cleaners']))[None, :]
    return torch.autograd.Variable(torch.from_numpy(sequence)).cuda().long()

def waveglow_inference(waveglow, denoiser, mel_outputs_postnet):
    with torch.no_grad():
            audio = waveglow.infer(mel_outputs_postnet, sigma=0.666)

    audio_denoised = denoiser(audio, strength=0.01)[:, 0]
    return audio_denoised.cpu().numpy()

def write_(audio, output_dir, sample_rate, sentence=None, filename=None):
    if filename:
        filepath = Path(output_dir) / f"{filename}.wav"
    elif sentence:
        filepath = prepare_output_filepath(output_dir, sentence)
    else:
        raise ValueError("Either `sentence` or `filename` parameters must be passed to write_() method")

    print(f"Writing {filepath}")
    write(filepath, sample_rate, audio[0])

def synthesize_text(taco_model, waveglow_model, denoiser, text: str, plot=False):
    sentences = prepare_sentences(text)
    return [synthesize_sentence(taco_model, waveglow_model, denoiser, sentence, plot) for sentence in sentences]

def synthesize_sentence(taco_model, waveglow_model, denoiser, sentence, plot=False):
    sequence = prepare_sequence(sentence)

    mel_outputs, mel_outputs_postnet, _, alignments = taco_model.inference(sequence)

    if plot:
        plot_data((mel_outputs.float().data.cpu().numpy()[0],
                   mel_outputs_postnet.float().data.cpu().numpy()[0],
                   alignments.float().data.cpu().numpy()[0].T))

    return waveglow_inference(waveglow_model, denoiser, mel_outputs_postnet)

def concatenate_audios(synthesized: List[ndarray], silence=0):
    synth_audios = synthesized.copy()
    if silence > 0:
        silent_frames = np.array(AudioSegment.silent(duration=1000).get_array_of_samples(), dtype=np.float32).reshape((1, -1))
        for i in reversed(range(len(synth_audios)-1)):
            synth_audios.insert(i, silent_frames)

    return np.concatenate(tuple(synth_audios), axis=1)


#### Setup hparams

In [46]:
output_dir = f"files/synthesis/aurimas-vocoder/aurimas-acoustic"
# output_name = f"mazasis_princas_aurimas"
# output_name = f"mazasis_princas_algis"
# output_name = f"mazasis_princas_sigutis"
# output_name = f"mazasis_princas_nijole"
# output_name = f"mazasis_princas_leonidas"
# output_name = f"mazasis_princas_paulius"
output_name = f"mazasis_princas_justas"
Path(output_dir).mkdir(parents=True, exist_ok=True)

# tacotron_path = "/media/arnas/SSD Disk/inovoice/models/text-to-speech/tacotron/aurimas-3/checkpoint_73000"
# waveglow_path = "/media/arnas/SSD Disk/inovoice/models/text-to-speech/waveglow/aurimas-2/waveglow_52000"

# tacotron_path = "/media/arnas/SSD Disk/inovoice/models/text-to-speech/tacotron/algis-stressed-1/checkpoint_24000"
# waveglow_path = "/media/arnas/SSD Disk/inovoice/models/text-to-speech/waveglow/algis-1/waveglow_12000"

# tacotron_path = "/media/arnas/SSD Disk/inovoice/models/text-to-speech/tacotron/sigutis-1/checkpoint_35000"
# waveglow_path = "/media/arnas/SSD Disk/inovoice/models/text-to-speech/waveglow/aurimas-2/waveglow_52000"

# tacotron_path = "/media/arnas/SSD Disk/inovoice/models/text-to-speech/tacotron/nijole-1/checkpoint_48000"
# waveglow_path = "/media/arnas/SSD Disk/inovoice/models/text-to-speech/waveglow/nijole-4/waveglow_30000"

# tacotron_path = "/media/arnas/SSD Disk/inovoice/models/text-to-speech/tacotron/leonidas-1/checkpoint_60000"
# waveglow_path = "/media/arnas/SSD Disk/inovoice/models/text-to-speech/waveglow/aurimas-2/waveglow_52000"

# tacotron_path = "/media/arnas/SSD Disk/inovoice/models/text-to-speech/tacotron/paulius-3/checkpoint_55000"
# waveglow_path = "/media/arnas/SSD Disk/inovoice/models/text-to-speech/waveglow/aurimas-2/waveglow_52000"

tacotron_path = "/media/arnas/SSD Disk/inovoice/models/text-to-speech/tacotron/justas-1/checkpoint_44000"
waveglow_path = "/media/arnas/SSD Disk/inovoice/models/text-to-speech/waveglow/aurimas-2/waveglow_52000"

hparams = create_hparams()
hparams.sampling_rate = 22050

#### Load Tacotron text2mel synthesis and WaveGlow for mel2audio synthesis and denoiser

In [47]:
tacotron = load_model(hparams)
tacotron.load_state_dict(torch.load(tacotron_path)['state_dict'])
_ = tacotron.cuda().eval().half()

waveglow = torch.load(waveglow_path)['model']
waveglow.cuda().eval().half()
for k in waveglow.convinv:
    k.float()
denoiser = Denoiser(waveglow)

In [48]:
karlsonas_text = """Astridà Lindgren.
MAŽỸLIS IR̃ KÁRLSONAS, KURÌS GYVẼNA AÑT STÓGO.
Ketvirtàsis leidìmas.
Kárlsonas, kurìs gyvẽna añt stógo Stòkholmo miestè, vienamè paprastõs gãtvės paprastamè namè kaĩp namè, gyvẽna paprastà švèdų Svantesonų šeimà.
Tojè šeimojè ràsite pàprastą tė̃tį, pàprastą mãmą ir̃ trìs visái pàprastus vaikùs - Bosę, Betaną ir̃ Mažỹlį.
- Õ àš nesù visái pàprastas mažỹlis, - sãko Mažỹlis.
Bèt taĩ, žìnoma, netiesà.
Jùk pasáulyje tíek daũg septynmẽčių mėlynãkių berniùkų sù neplautõm ausim̃ ir̃ kiauraĩ pratrintaĩs kélnių kẽliais, kàd nėrà kõ nė̃ abejóti: Mažỹlis visái pàprastas berniùkas.
Bosei - penkiólika mẽtų, ir̃ jìs daũg mieliaũ stóvi var̃tuose fùtbolo aikštėjè negù priẽ lentõs klãsėje, taĩgi jìs yrà visái pàprastas berniùkas.
Betanai - keturiólika mẽtų, ir̃ jõs kãsos vìsiškai tókios pàt kaĩp ir̃ kitų̃ paprastų̃ mergáičių.
Ir̃ visamè namè yrà tìk vienà nè visái paprastà būtýbė Kárlsonas, kurìs gyvẽna añt stógo.
Tikraĩ, jìs gyvẽna añt stógo, ir̃ jaũ víen dė̃l tõ yrà nè visái pàprastas.
Gál kituosè miẽstuose bū̃na kitaĩp, bèt Stòkholme bevéik niẽkad niẽkas negyvẽno añt stógo, ir̃ dár atskiramè mažame.namelyje.
Õ šitasaĩ Kárlsonas, tìk pagalvókite, gyvẽna kaĩp tìk tenaĩ.
Kárlsonas - taĩ mažýtis, stóras, išdidùs žmogùtis, ir̃ dár jìs móka skraidýti.
Lėktùvais ir̃ malū̃nsparniais skraidýti visì gùdrūs, õ Kárlsonas pàts skreñda.
Võs tìk paspáudžia mygtùką añt sàvo pil̃vo, añt nùgaros tuõj pràdeda sùktis mãžas motoriùkas.
Kõl propèleris geraĩ neįsivãręs, Kárlsonas ramiaĩ stóvi, bèt kaĩ motòras įsibėgė́ja, Kárlsonas šáuna aukštỹn ir̃ lẽkia trupùtį svirduliúodamas, tóks pasipū̃tęs ir̃ orùs, tar̃tum bū́tų kóks dirèktorius, - suprañtama, jéi tìk įsivaizdúosime dirèktorių sù propèleriu añt nùgaros.
Kárlsonas gražiaĩ gyvẽna mažamè namẽlyje añt stógo.
Vakaraĩs jìs sė́di priebutė̃lyje, pàpsi sáu pýpkę ir̃ žiū̃ri į̃ žvaigždès. Žìnoma, nuõ stógo žvaĩgždės matýti aiškiaũ negù prõ lángus, taĩgi beliẽka tìk stebė́tis, kodė̃l taĩp mažaĩ kàs gyvẽna añt stogų̃.
Matýt, kitíems nė̃ nèmaga įsikùrti añt stógo.
Jùk jiẽ nežìno, kàd teñ Kárlsonas tùri sàvo namẽlį, nès tàs namẽlis užsiglaũdęs ùž dìdelio kãmino.
Õ pagaliaũ ar̃gi suáugusiems rūpė̃s kóks mažýtis nameliùkas, nèt jéigu ir̃ užsidùrtų añt jõ?
Sỹkį víenas kaminkrėtỹs staigà pamãtė Kárlsono namẽlį.
Jìs labaĩ nusistebė́jo ir̃ tãrė sáu: - Keĩsta...
Namẽlis?..
Negãli bū́ti!
Añt stógo stóvi mažýtis namẽlis?..
Kaĩp jìs galė́jo čià atsiràsti?
Paskuĩ kaminkrėtỹs ė̃mė krė̃sti kãminą, užmir̃šo namẽlį ir̃ daugiaũ jõ nebeprisìminė.
Mažỹlis labaĩ džiaũgėsi susipažìnęs sù Kárlsonu.
Kaĩ tiktaĩ Kárlsonas atskrìsdavo, prasidė́davo nepaprastì núotykiai.
Kárlsonui, matýt, ir̃gi bùvo malonù sueĩti į̃ pãžintį sù Mažyliù.
Ką̃ nóri sakýk, õ nelabaĩ jaukù vienuĩ vienám gyvénti mažamè namẽlyje, ir̃ dár tokiamè, apiẽ kurį̃ niẽkas nėrà nė̃ girdė́jęs.
Liū̃dna, jéi nėrà kám sušùkti: \"Sveĩkas, Kárlsonai!\", kadà tù skrendì prõ šãlį.
Jų̃ pažintìs prasidė́jo víeną tókią nelaimìngą diẽną, kaĩ Mažỹlis nèjautė jókio džiaũgsmo, kàd jìs mažỹlis, nórs šiaĩp tokiám bū́ti labaĩ smagù.
Jùk Mažỹlis - visõs šeimõs numylė́tinis, ir̃ kiekvíenas jį̃ paĩkina kaĩp išmanýdamas.
Bèt tą̃ diẽną vìskas ė̃jo atbulaĩ.
Mamà jį̃ ìšbarė, kàd vė̃l pérsiplėšė kélnes, Betana sušùko: \"Nusišlúostyk nósį!\", õ tė̃tis užsirū́stino añt Mažỹlio, kàd taĩp vėlaĩ sugrį̃žo ìš mokỹklos.
- Gãtvėmis šlaistaĩsi! - pasãkė tė̃tis.
\"Gãtvėmis šlaistaĩsi!\" Bèt jùk tė̃tis nežinójo, kàd eĩnant namõ Mažỹliui pasipáiniojo šunýtis.
Miẽlas, puikùs šunýtis: jìs apúostė Mažỹlį ir̃ maloniaĩ suvìzgino úodegą lýg sakýdamas, kàd nóri bū́ti jõ šunýtis.
Jéigu bū́tų bùvusi Mažỹlio valià, šunýčio nórą bū́tų tuõj paténkinęs.
Bèt visà bėdà, kàd mamà ir̃ tė̃tis niekù būdù nenorė́jo laikýti namuosè šuñs.
Õ teñ dár ùž kam̃po staigà išnìro kažkokià tetà ir̃ sušùko: \"Rìki!
Rìki!
Eĩk šen!\" - ir̃ tadà Mažỹliui pasidãrė vìsiškai áišku, kàd šìtas šunýtis jaũ niekadõs nebùs jõ šunýtis.
- Atródo, kàd taĩp vìsą gyvẽnimą ir̃ nugyvénsi bè šuñs, graũdžiai tãrė Mažỹlis, kaĩ vìskas taĩp liūdnaĩ baĩgėsi. - Tù, mãma, turì tė̃tį; ir̃ Bosè sù Betana taĩp pàt visadõs kartù.
Õ àš - àš niẽko neturiù!..
- Brangùs Mažỹli, jùk mẽs visì tàvo! - pasãkė mamà.
- Nežinaũ... - dár graudžiaũ sulemẽno Mažỹlis, nès jám staigà pasiródė, kàd jìs ìš tiesų̃ niẽko netùri pasáulyje.
Bejè, jìs turė́jo sàvo kam̃barį ir̃ dabar̃ nuė̃jo į̃ jį̃.
Bùvo vaiskùs pavãsario vãkaras, visì langaĩ atdarì.
"""

In [49]:
mazasis_princas_text="""
Antuãnas de Sént-Egziuperì.
MAŽÀSIS PRÌNCAS.
Dedikãcija LẼONUI VERTUI.
Atsiprašaũ vaikùs, kàd šią̃ knỹgą paskýriau suáugusiam.
Turiù rim̃tą pasitéisinimą: tàs žmogùs yrà geriáusias pasáulyje màno draũgas.
Turiù dár kìtą pasitéisinimą: tàs žmogùs gãli vìską supràsti, nèt knygàs vaikáms.
Turiù ir̃ trẽčią pasitéisinimą: tàs žmogùs gyvẽna Prancūzìjoje, jìs čià álksta ir̃ šą̃la.
Jám labaĩ reĩkia pagúodos. Jéi šių̃ visų̃ pasitéisinimų neužteñka, taĩ šią̃ knỹgą skiriù vaĩkui, kurìs kitadõs bùvo suáugęs žmogùs.
Visì suáugusieji ìš pradžių̃ bùvo vaikaĩ. (Tìk nedaũgelis taĩ atsìmena).
Taĩgi pataisaũ dedikãciją LẼONUI VERTUI.
KAĨ JÌS BÙVO MÃŽAS BERNIÙKAS.
Pìrmas skỹrius - mẽs supažìndinami sù pãsakotoju, lakūnù, ir̃ jõ idė́jomis apiẽ suáugusius.
Pìrmas skỹrius.
Kaĩ mán bùvo šešerì mẽtai, kar̃tą vienojè knỹgoje apiẽ Neįžeñgiamą gìrią - knygà vadìnosi \"Pãsakojimai apiẽ dalykùs, kuriẽ tikraĩ įvyko\",- radaũ puĩkų paveiksliùką.
Jamè bùvo atvaizdúotas smauglỹs, rỹjantis žvė́rį.
Štaĩ čià pérpieštas tàs paveiksliùkas.
Knỹgoje bùvo parašýta:
Smaugliaĩ surỹja sàvo grõbį ìštisą, jõ nekram̃tę.
Paskuĩ jiẽ nebegãli judė́ti ir̃ šešìs mė́nesius miẽga, kõl vìską suvìrškina.
Àš tadà daũg galvójau apiẽ núotykius džiùnglėse, ir̃ mán ir̃gi pavỹko nupiẽšti spalvótu pieštukù pìrmąjį sàvo piẽšinį.
Sàvo piẽšinį nùmeris víenas. Jìs bùvo šìtoks:
Sàvo kū́rinį àš paródžiau suáugusiems ir̃ pakláusiau, ar̃ jíems màno piẽšinio baisù.
Jiẽ mán atsãkė: \"Õ kõ čià bùs baisù skrybėlė̃s?\"
Àš buvaũ nupiẽšęs visái nè skrýbėlę.
Buvaũ nupiẽšęs smaũglį, vìrškinantį dram̃blį.
Tadà nùpiešiau smaũglio vìdų, kàd suáugusiems bū́tų suprantamiaũ.
Jíems visadà reĩkia áiškinti.
Màno piešinỹs nùmeris dù bùvo šìtoks:
Suáugę žmónės mán pàtarė liáutis piẽšti ãtvirus ar̃ ùždarus smaugliùs ir̃ verčiaũ domė́tis geogrãfija, istòrija, skaičiãvimu ir̃ gramãtika.
Taĩp, bū́damas šešerių̃ mẽtų atsisakiaũ puikiõs daĩlininko karjèros.
Mán sugadìno núotaiką taĩ, kàd màno piešiniaĩ nùmeris víenas ir̃ nùmeris dù neturė́jo pasisekìmo.
Suáugusieji niekadà niẽko nesuprañta pãtys, ir̃ vaikáms tìkras var̃gas, kàd tùri jíems vìs áiškinti ir̃ áiškinti.
Taĩgi mán tẽko riñktis kitókią profèsiją, ir̃ àš išmókau skraidýti lėktùvais. Skraidžiaũ ir̃ šeñ, ir̃ teñ põ pasáulį.
Ir̃ geogrãfija mán labaĩ pràvertė.
Galė́davau ìš pìrmo žvil̃gsnio atskìrti Kìniją nuõ Arizònos.
Taĩ išeĩna labaĩ į̃ gẽrą, jéi nakčià pasiklýsti.
Añtras skỹrius - pãsakotojo lėktùvas sudū̃žta dykumojè ir̃ jìs susipažį́sta sù mažúoju prìncu.
Añtras skỹrius.
Taĩp ir̃ gyvenaũ víenas, neturė́damas sù kuõ tikraĩ pasišnekė́ti, kõl priẽš šẽšerius metùs Sachãroje sugẽdo màno lėktùvas.
Jõ motòre kažkàs sulū́žo.
Ir̃ kadángi sù manim̃ nebùvo neĩ mechãniko, neĩ keleĩvių, nùtariau pabandýti pàts pašãlinti nemeñką gedìmą. Nuõ tõ priklaũsė, ar̃ lìksiu gývas, ar̃ nè.
Vandeñs teturė́jau võs kókiai saváitei.
Taĩgi pìrmąjį vãkarą užmigaũ añt smė̃lio, ùž tū́kstančio mỹlių nuõ bèt kuriõs gyvẽnamos viẽtos.
Buvaũ labiaũ nuõ visų̃ atskìrtas negù žmogùs, laĩvui sudùžus, kláidžiojantis plaustù vidurỹ vandenýno.
Todė̃l gãlite įsivaizdúoti, kaĩp nustebaũ, kaĩ apýaušriu manè pažãdino kažkóks keĩstas balsẽlis.
Kažkàs tãrė:
- Prašaũ... nupiẽšk mán ãvį!
- À?
- Nupiẽšk mán ãvį..."""

In [50]:
audios = synthesize_text(tacotron, waveglow, denoiser, mazasis_princas_text, plot=True)
audio = concatenate_audios(audios, silence=0.5)

Warning! Reached max decoder steps
Warning! Reached max decoder steps
Warning! Reached max decoder steps


In [51]:
write_(audio, output_dir, hparams.sampling_rate, filename=output_name)

Writing files/synthesis/aurimas-vocoder/aurimas-acoustic/mazasis_princas_justas.wav
